# Test combination of PCA and Normalizing Flow

In [ ]:
# Import packages
import numpy as np
import torch
import normflow as nf
import larsflow as lf

from sklearn.datasets import make_moons

from matplotlib import pyplot as plt

from tqdm import tqdm

In [ ]:
# Set up model

# Define flows
K = 8
torch.manual_seed(0)

latent_size = 2
b = torch.Tensor([1 if i % 2 == 0 else 0 for i in range(latent_size)])
flows = []
for i in range(K):
    s = nf.nets.MLP([latent_size, 32, 32, latent_size], init_zeros=True)#, output_fn="tanh", output_scale=3.)
    t = nf.nets.MLP([latent_size, 32, 32, latent_size], init_zeros=True)#, output_fn="tanh", output_scale=3.)
    if i % 2 == 0:
        flows += [nf.flows.MaskedAffineFlow(b, s, t)]
    else:
        flows += [nf.flows.MaskedAffineFlow(1 - b, s, t)]
    flows += [nf.flows.ActNorm(latent_size)]

# Set prior and q0
a = nf.nets.MLP([latent_size, 128, 128, 128, 1], output_fn="sigmoid", leaky=0.01)
q0 = lf.distributions.ResampledGaussian(latent_size, a, 100, 0.1, trainable=False)
#q0 = nf.distributions.DiagGaussian(latent_size)

# Target distribution
p = nf.distributions.TwoModes(2, 0.1)
#img = 1 - plt.imread('/var/home/vs488/Downloads/smiley.png')[:, :, 2]
#img = np.pad(img, ((1, 1), (1, 1)))
#p = nf.distributions.ImagePrior(img, x_range=[-2, 2], y_range=[-2, 2])

# Construct flow model
nfm = lf.NormalizingFlow(q0=q0, flows=flows, p=p)

# Move model on GPU if available
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')
nfm = nfm.to(device)
nfm = nfm.double()

# Initialize ActNorm
z, _ = nfm.sample(num_samples=2 ** 14)
z_np = z.to('cpu').data.numpy()
plt.figure(figsize=(15, 15))
plt.hist2d(z_np[:, 0].flatten(), z_np[:, 1].flatten(), (200, 200), range=[[-3, 3], [-3, 3]])
plt.gca().set_aspect('equal', 'box')
plt.show()

In [ ]:
# Plot target distribution
grid_size = 300
xx, yy = torch.meshgrid(torch.linspace(-3, 3, grid_size), torch.linspace(-3, 3, grid_size))
zz = torch.cat([xx.unsqueeze(2), yy.unsqueeze(2)], 2).view(-1, 2)
zz = zz.double().to(device)

#x_np, _ = make_moons(2 ** 15, noise=.05)
#plt.figure(figsize=(15, 15))
#plt.hist2d(x_np[:, 0], x_np[:, 1], (grid_size, grid_size), range=[[-3, 3], [-3, 3]])
#plt.savefig('pca_test_target.png')
#plt.show()

log_prob = nfm.p.log_prob(zz).to('cpu').view(*xx.shape)
prob = torch.exp(log_prob)
prob[torch.isnan(prob)] = 0

plt.figure(figsize=(15, 15))
plt.pcolormesh(xx, yy, prob.data.numpy())
plt.gca().set_aspect('equal', 'box')
plt.show()

# Plot initial flow distribution
nfm.eval()
log_prob = nfm.log_prob(zz).to('cpu').view(*xx.shape)
prob = torch.exp(log_prob)
prob[torch.isnan(prob)] = 0

plt.figure(figsize=(15, 15))
plt.pcolormesh(xx, yy, prob.data.numpy())
plt.gca().set_aspect('equal', 'box')
#plt.savefig('pca_test_init_flow.png')
plt.show()

# Plot initial base distribution
log_prob = nfm.q0.log_prob(zz).to('cpu').view(*xx.shape)
prob = torch.exp(log_prob)
prob[torch.isnan(prob)] = 0

plt.figure(figsize=(15, 15))
plt.pcolormesh(xx, yy, prob.data.numpy())
plt.gca().set_aspect('equal', 'box')
#plt.savefig('pca_test_init_base.png')
plt.show()

In [ ]:
# Train model
max_iter = 6000
num_samples = 2 ** 10
show_iter = 1000

lr_decay = 0.1
lr_decay_iter = 100000

loss_hist = np.array([])
Z_hist = np.array([])

nfm.train()

optimizer = torch.optim.Adam(nfm.parameters(), lr=1e-3, weight_decay=1e-5)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer,
                                                      gamma=lr_decay)
for it in tqdm(range(max_iter)):
    optimizer.zero_grad()
    
    #x_np, _ = make_moons(num_samples, noise=.05)
    #x = torch.tensor(x_np).to(device)
    #x = nfm.p.sample(num_samples).double()
    
    #loss = nfm.forward_kld(x) #- np.max([0, 0.5 * (it / 15000 - 1)]) * nfm.q0.Z
    #loss = nfm.reverse_kld(num_samples, beta=np.min([1, it / 5000 + 0.01]))
    loss = nfm.reverse_kld_cov(num_samples, beta=np.min([1, np.max([0, (it - 0) / 3000 + 0.01])]))
    
    if ~(torch.isnan(loss) | torch.isinf(loss)):
        loss.backward()
        optimizer.step()
    
    loss_hist = np.append(loss_hist, loss.to('cpu').data.numpy())
    Z_hist = np.append(Z_hist, nfm.q0.Z.to('cpu').data.numpy())
    
    # Plot learned posterior
    if (it + 1) % show_iter == 0:
        nfm.eval()
        log_prob = nfm.log_prob(zz).to('cpu').view(*xx.shape)
            
        prob = torch.exp(log_prob.to('cpu').view(*xx.shape))
        prob[torch.isnan(prob)] = 0
        
        log_prob = p.log_prob(zz).to('cpu').view(*xx.shape)
        prob_p = torch.exp(log_prob)

        plt.figure(figsize=(15, 15))
        plt.pcolormesh(xx, yy, prob.data.numpy())
        plt.contour(xx, yy, prob_p.data.numpy(), cmap=plt.get_cmap('cool'), linewidths=2)
        plt.gca().set_aspect('equal', 'box')
        plt.show()
        
        log_prob = nfm.q0.log_prob(zz).to('cpu').view(*xx.shape)
        prob = torch.exp(log_prob)
        prob[torch.isnan(prob)] = 0

        plt.figure(figsize=(15, 15))
        plt.pcolormesh(xx, yy, prob.data.numpy())
        plt.gca().set_aspect('equal', 'box')
        plt.show()
        
        prob = nfm.q0.a(zz).to('cpu').view(*xx.shape)
        prob[torch.isnan(prob)] = 0

        plt.figure(figsize=(15, 15))
        plt.pcolormesh(xx, yy, prob.data.numpy())
        plt.gca().set_aspect('equal', 'box')
        plt.colorbar()
        plt.show()
        nfm.train()
    
    if (it + 1) % lr_decay_iter == 0:
        lr_scheduler.step()
        

plt.figure(figsize=(10, 10))
plt.plot(loss_hist, label='loss')
plt.legend()
plt.show()

plt.figure(figsize=(10, 10))
plt.plot(Z_hist, label='Z')
plt.legend()
plt.show()

In [ ]:
# Plot flow distribution after training
nfm.eval()
log_prob = nfm.log_prob(zz).to('cpu').view(*xx.shape)
prob = torch.exp(log_prob)
prob[torch.isnan(prob)] = 0
log_prob = p.log_prob(zz).to('cpu').view(*xx.shape)
prob_p = torch.exp(log_prob)

plt.figure(figsize=(15, 15))
plt.pcolormesh(xx, yy, prob.data.numpy())
plt.contour(xx, yy, prob_p.data.numpy(), cmap=plt.get_cmap('cool'), linewidths=2)
plt.gca().set_aspect('equal', 'box')
plt.savefig('resampled_final.png')
plt.show()

# Plot base distribution after training
log_prob = nfm.q0.log_prob(zz).to('cpu').view(*xx.shape)
prob = torch.exp(log_prob)
prob[torch.isnan(prob)] = 0

plt.figure(figsize=(15, 15))
plt.pcolormesh(xx, yy, prob.data.numpy())
plt.gca().set_aspect('equal', 'box')
plt.savefig('resampled_base_final.png')
plt.show()

# Plot acceptance probability after training
prob = nfm.q0.a(zz).to('cpu').view(*xx.shape)
prob[torch.isnan(prob)] = 0

plt.figure(figsize=(15, 15))
plt.pcolormesh(xx, yy, prob.data.numpy())
plt.gca().set_aspect('equal', 'box')
plt.colorbar()
plt.savefig('a_final.png')
plt.show()

# Plot histogram of samples
z, _ = nfm.sample(num_samples=2 ** 12)
z_np = z.to('cpu').data.numpy()
plt.figure(figsize=(15, 15))
plt.hist2d(z_np[:, 0].flatten(), z_np[:, 1].flatten(), (200, 200), range=[[-3, 3], [-3, 3]])
plt.gca().set_aspect('equal', 'box')
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(np.log(Z_hist), label='Z')
plt.legend()
plt.show()